# Курсовой проект для курса "Python для Data Science. Продолжение."
#### Исполнитель Васильев А.

### Задача
Требуется, на основании имеющихся данных о клиентах банка, построить модель, используя обучающий датасет, для прогнозирования невыполнения долговых обязательств по текущему кредиту. Выполнить прогноз для примеров из тестового датасета.

In [ ]:
import numpy as np
import pandas as pd
import pickle
import random

from scipy.stats import shapiro
from scipy.stats import probplot
from scipy.stats import ttest_ind, mannwhitneyu
from scipy.stats import chi2_contingency
from statsmodels.stats.weightstats import zconfint

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, learning_curve
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import lightgbm as lgbm
import catboost as catb

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
pd.options.display.max_columns = 100

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_classification_report(y_train_true, y_train_pred, y_valid_true, y_valid_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('VALID\n\n' + classification_report(y_valid_true, y_valid_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_valid_true, y_valid_pred))

In [ ]:
def balance_df_by_target(df, target_name):

    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1

    for i in range(disbalance_coeff):
        sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
        df = df.append(sample, ignore_index=True)

    return df.sample(frac=1) 

In [ ]:
def get_feature_importances(feature_names, feature_importances, get_top=None):
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
    feature_importances = feature_importances.sort_values('importance', ascending=False)
    
    if get_top is not None:
        return feature_importances['feature'][:get_top].tolist()

**Описание датасета**

* **Home Ownership** - домовладение
* **Annual Income** - годовой доход
* **Years in current job** - количество лет на текущем месте работы
* **Tax Liens** - налоговые льготы
* **Number of Open Accounts** - количество открытых счетов
* **Years of Credit History** - количество лет кредитной истории
* **Maximum Open Credit** - наибольший открытый кредит
* **Number of Credit Problems** - количество проблем с кредитом
* **Months since last delinquent** - количество месяцев с последней просрочки платежа
* **Bankruptcies** - банкротства
* **Purpose** - цель кредита
* **Term** - срок кредита
* **Current Loan Amount** - текущая сумма кредита
* **Current Credit Balance** - текущий кредитный баланс
* **Monthly Debt** - ежемесячный долг
* **Credit Score** - оценка благонадежности клиента (скоринговый балл, полученный из другого источника)
* **Credit Default** - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [ ]:
TRAIN_DATASET_PATH = 'datasets/course_project_train.csv'
TEST_DATASET_PATH = 'datasets/course_project_test.csv'

PREDICTED_CREDIT_DEFAULT_PATH  = 'AVasilev_predictions.csv'

MODEL_FILE_PATH = 'model.pkl'

#### Обзор обучающего датасета

In [ ]:
df_train = pd.read_csv(TRAIN_DATASET_PATH)
df_train.shape

In [ ]:
df_train.head()

In [ ]:
df_train['Credit Default'].value_counts(normalize=True)

In [ ]:
df_train.info()

In [ ]:
print('Количество не заполненных значений:')
df_train.isnull().sum()

### Обработка выбросов

**Обучающий датасет**

In [ ]:
df_train.describe()

*Maximum Open Credit*

In [ ]:
median_open_credit = df_train['Maximum Open Credit'].median()
df_train.loc[df_train['Maximum Open Credit'] > 1e+07, 'Maximum Open Credit'] = median_open_credit

*Current Loan Amount*

In [ ]:
median_loan_ammount = df_train['Current Loan Amount'].median()
df_train.loc[df_train['Current Loan Amount'] > 1e+06, 'Current Loan Amount'] = median_loan_ammount

*Credit Score*

In [ ]:
df_train.loc[df_train['Credit Score'] > 751, 'Credit Score'] = df_train.loc[df_train['Credit Score'] > 751, 'Credit Score'] / 10

### Обработка пропусков обучающего датасета

##### Annual Income

In [ ]:
for i in df_train.index[df_train['Annual Income'].isnull()]:
    monthly_debt = df_train.iloc[i, 14]
    df_train.iat[i, 1] = df_train.loc[(df_train['Monthly Debt'] < monthly_debt * 1.2) & 
                                      (df_train['Monthly Debt'] > monthly_debt * 0.8), 'Monthly Debt'].median()

In [ ]:
med_anual_income = df_train['Annual Income'].median()
df_train.loc[df_train['Annual Income'].isnull(), 'Annual Income'] = med_anual_income

##### Years in current job

In [ ]:
job_years = df_train['Years in current job'].mode()[0]
df_train.loc[df_train['Years in current job'].isnull(), 'Years in current job'] = job_years

##### Credit Score

In [ ]:
credit_score = {}
for value in df_train['Years of Credit History'].unique():
    credit_score[value] = df_train.loc[df_train['Years of Credit History'] == value, 'Credit Score'].median()

In [ ]:
for i in df_train.index[df_train['Credit Score'].isnull()]:
    cred_history_years = df_train.iloc[i, 5]
    df_train.iat[i, 15] = credit_score[cred_history_years]

In [ ]:
median_credit_score = df_train['Credit Score'].median()
df_train.loc[df_train['Credit Score'].isnull(), 'Credit Score'] = median_credit_score

max_credit_score = df_train['Credit Score'].max()
min_credit_score = df_train['Credit Score'].min()

df_train.loc[df_train['Credit Score'].isnull() & (df_train['Credit Default'] == 0), 'Credit Score'] = min_credit_score

df_train.loc[df_train['Credit Score'].isnull(), 'Credit Score'] = max_credit_score

###### для последующего заполнения пропусков в тестово датасете
credit_score = {}
for value in df_train['Years of Credit History'].unique():
    credit_score[value] = df_train.loc[df_train['Years of Credit History'] == value, 'Credit Score'].median()

median_credit_score = df_train['Credit Score'].median()

##### Bankruptcies

In [ ]:
bankruptcies = {}
for value in df_train['Credit Score'].unique():
    bankruptcies[value] = df_train.loc[df_train['Credit Score'] == value, 'Bankruptcies'].median()

In [ ]:
for i in df_train.index[df_train['Bankruptcies'].isnull()]:
    cred_score = df_train.iloc[i, 15]
    df_train.iat[i, 9] = bankruptcies[cred_score]

##### Months since last delinquent

Т.к. пропущенных значений больше половины, удаляю данный признак из датасета

In [ ]:
df_train = df_train.drop(columns='Months since last delinquent')

In [ ]:
df_train.isnull().sum()

### Создаю дополнительные признаки

**Is Loan Big - равен 1, если текущая сумма кредита больше медианного значения**

In [ ]:
med_cur_ammount = df_train['Current Loan Amount'].median()
is_big_loan = [1 if el > med_cur_ammount else 0 for el in df_train['Current Loan Amount']]
df_train['Is Loan Big'] = is_big_loan

**Mean Home Ownership Income - средний доход домовладений по типам**

In [ ]:
home_own_types = df_train['Home Ownership'].unique()

In [ ]:
mean_home_own_income_by_type = [df_train.loc[df_train['Home Ownership'] == el, 'Annual Income'].mean() 
                                for el in home_own_types]

In [ ]:
df_mean_income = pd.DataFrame([home_own_types, mean_home_own_income_by_type], index=['Home Ownership', 'Mean Annual Income'])
df_mean_income = df_mean_income.T

In [ ]:
mean_home_own_income_temp = [df_mean_income.loc[df_mean_income['Home Ownership'] == el, 'Mean Annual Income'].tolist()
                             for el in df_train['Home Ownership']]

mean_home_own_income = [el[0] for el in mean_home_own_income_temp]

In [ ]:
df_train['Mean Home Ownership Income'] = mean_home_own_income

**Debt-to-income ratio - Отношение ежемесячного долга домовладения к его ежемесячному доходу**

In [ ]:
df_train['Debt-to-income ratio'] = round(df_train['Monthly Debt'] / (df_train['Annual Income'] / 12), 3)

**Is Payment High - Равен 1, если Debt-to-income ratio более 0.43.**  
Считается, что при более высокой доле обязательств по кредиту относительно общего дохода, становится затруднительно отвечать по своим финансовым обязательствам. Данное обстоятельство ведет к повышению риска просрочки платежей по ним.

In [ ]:
is_payment_high = [1 if el > 0.43 else 0 for el in df_train['Debt-to-income ratio']]
is_payment_high[:5]

In [ ]:
df_train['Is Payment High'] = is_payment_high

### Обзор категориальных переменных

#### Обучающий датасет

In [ ]:
for cat_colname in df_train.select_dtypes(include='object').columns:
    print(str(cat_colname) + '\n\n' + str(df_train[cat_colname].value_counts()) + '\n' + '*' * 100 + '\n')

In [ ]:
df_train['Term Binary'] = df_train['Term'].map({'Short Term':'0', 'Long Term':'1'}).astype(int)

In [ ]:
df_train['Home Ownership digit'] = df_train['Home Ownership'].map({'Home Mortgage':'0', 'Rent':'1',\
                                    'Own Home': '2', 'Have Mortgage': '3'}).astype(int)

df_train.loc[(df_train['Years in current job'] == '< 1 year') | (df_train['Years in current job'] == '1 year'), 'Years in current job'] = 1
df_train.loc[(df_train['Years in current job'] == '2 years') | (df_train['Years in current job'] == '3 years'), 'Years in current job'] = 2
df_train.loc[(df_train['Years in current job'] == '4 years') | (df_train['Years in current job'] == '5 years'), 'Years in current job'] = 3
df_train.loc[(df_train['Years in current job'] == '6 years') | (df_train['Years in current job'] == '7 years'), 'Years in current job'] = 4
df_train.loc[(df_train['Years in current job'] == '8 years') | (df_train['Years in current job'] == '9 years'), 'Years in current job'] = 5
df_train.loc[df_train['Years in current job'] == '10+ years', 'Years in current job'] = 6

In [ ]:
obj_columns = ['Purpose', 'Years in current job']

In [ ]:
train_dummies = pd.get_dummies(df_train[obj_columns])
df_train = pd.concat([df_train, train_dummies], axis=1)

In [ ]:
df_train.shape

### Анализ целевой переменной

In [ ]:
df_base = pd.read_csv(TRAIN_DATASET_PATH)

In [ ]:
df_base.head()

In [ ]:
TARGET_NAME = 'Credit Default'
BASE_FEATURE_NAMES = df_base.columns.drop(['Months since last delinquent', 'Credit Default']).tolist()
BASE_FEATURE_NAMES.append('Is Loan Big')
BASE_FEATURE_NAMES.append('Mean Home Ownership Income')
BASE_FEATURE_NAMES.append('Debt-to-income ratio')
BASE_FEATURE_NAMES.append('Is Payment High')
NEW_FEATURE_NAMES = df_train.columns.drop([TARGET_NAME] + BASE_FEATURE_NAMES).tolist()

**Обзор распределения**

In [ ]:
df_train[TARGET_NAME].value_counts()

In [ ]:
plt.figure(figsize=(8, 5))

sns.countplot(x=TARGET_NAME, data=df_train)

plt.title('Target variable distribution')
plt.show()

**Корреляция с базовыми признаками**

In [ ]:
corr_with_target = df_train[BASE_FEATURE_NAMES + [TARGET_NAME]].corr().iloc[:-1, -1].sort_values(ascending=False)

plt.figure(figsize=(10, 8))

sns.barplot(x=corr_with_target.values, y=corr_with_target.index)

plt.title('Correlation with target variable')
plt.show()

На основании этого графика, принял решение произвести оценку признаков с корреляцией более 0.01 по модулю.

**Оценка признака "Credit Score" в разрезе целевой переменной**

In [ ]:
credit_score_with_target_s = df_train[['Credit Score', TARGET_NAME]].sample(2000)
credit_score_s = credit_score_with_target_s['Credit Score']
credit_score_target_0 = credit_score_s[credit_score_with_target_s[TARGET_NAME] == 0]
credit_score_target_1 = credit_score_s[credit_score_with_target_s[TARGET_NAME] == 1]

plt.figure(figsize=(10, 5))

sns.kdeplot(credit_score_target_0, shade=True, label='No default', color='g')
sns.kdeplot(credit_score_target_1, shade=True, label='Default', color='r')

plt.xlabel('Credit Score')
plt.title('Credit Score grouped by target variable')
plt.show()

**Наблюдение**  
Есть вероятность, что между признаком "Credit Score" и целевой переменной существует некоторая функциональная зависимость.

**Гипотеза** 
* Нулевая гипотеза: средние значения в двух независимых выборках равны
* Альтернативная гипотеза: средние значения в двух независимых выборках различаются
* Критерий: критерий Стьюдента (t-тест)
* Уровень значимости $\normalsize \alpha$: 0.05
* Критическая область: двухсторонняя

*Проверка распределения признака на "нормальность" с помощью критерия Шапиро-Уилка*

In [ ]:
shapiro(credit_score_s)

Так как значение p-value равно 0, можно сделать вывод, что гипотеза о "нормальности" отвергается.

*Для достоверности провожу визуальную оценку распределения признака, а так же сторою QQ-график*

In [ ]:
plt.figure(figsize=(12, 6))

ax1 = plt.subplot(121)
ax1.set_xlabel('Credit Score')
ax1.set_ylabel('Count')
ax1.set_title('Credit Score distribution')
credit_score_s.hist()

plt.subplot(122)
probplot(credit_score_s, dist='norm', plot=plt)

plt.show()

*Оцениваю эквивалентность мат. ожиданий в исследуемых группах с помощью критерия Манна-Уитни.*

In [ ]:
mannwhitneyu(credit_score_target_0, credit_score_target_1)

Согласно значению p-value, гипотеза о равности мат. ожиданий отвергается.

*Строю доверительные интервалы для средних значений каждой из двух групп и сравним их для дополнительной проверки.*

In [ ]:
plt.figure(figsize=(8, 5))

sns.pointplot(x=TARGET_NAME, y='Credit Score', data=credit_score_with_target_s, capsize=.1)

plt.title('Confidence intervals (95 %) for Credit Score')
plt.show()

График подтверждает, что интервалы не пересекаются, что подтверждает результаты полученные с помощью критерия Манна-Уитни.

Полученные разельтаты означают, что группы из которых взяты данные выборки, с допускаемой вероятностью 95%, имеют различные распределения и этот признак может быть полезен для определения значения целевой переменной.

**Оценка признака "Annual Income" в разрезе целевой переменной**

In [ ]:
annual_income_with_target_s = df_train[['Annual Income', TARGET_NAME]].sample(1000)
annual_income_s = annual_income_with_target_s['Annual Income']
annual_income_target_0 = annual_income_s[annual_income_with_target_s[TARGET_NAME] == 0]
annual_income_target_1 = annual_income_s[annual_income_with_target_s[TARGET_NAME] == 1]

**Наблюдение**  
Есть вероятность, что между признаком "Annual Income" и целевой переменной существует некоторая функциональная зависимость.

**Гипотеза** 
* Нулевая гипотеза: средние значения в двух независимых выборках равны
* Альтернативная гипотеза: средние значения в двух независимых выборках различаются
* Критерий: критерий Стьюдента (t-тест)
* Уровень значимости $\normalsize \alpha$: 0.05
* Критическая область: двухсторонняя

*Проверка распределения признака на "нормальность" с помощью критерия Шапиро-Уилка*

In [ ]:
shapiro(annual_income_s)

Так как значение p-value сильно меньше 0.05, можно сделать вывод, что гипотеза о "нормальности" отвергается.

*Оцениваю эквивалентность мат. ожиданий в исследуемых группах с помощью критерия Манна-Уитни.*

In [ ]:
mannwhitneyu(annual_income_target_0, annual_income_target_1)

Согласно значению p-value, гипотеза о равности мат. ожиданий отвергается.  
Полученные разельтаты означают, что группы из которых взяты данные выборки, с допускаемой вероятностью 95%, имеют различные распределения и этот признак может быть полезен для определения значения целевой переменной.

### Анализ признакового пространства

**Матрица корреляций**

In [ ]:
plt.figure(figsize = (12,8))

sns.set(font_scale=0.5)
sns.heatmap(df_train[BASE_FEATURE_NAMES].corr().round(3), annot=True, linewidths=.5, cmap='GnBu')

plt.title('Correlation matrix')
plt.show()

### Отбор признаков

In [ ]:
NUM_FEATURE_NAMES = ['Annual Income', 'Tax Liens', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit',
                     'Number of Credit Problems', 'Bankruptcies', 'Current Loan Amount', 'Current Credit Balance', 
                     'Monthly Debt', 'Credit Score', 'Mean Home Ownership Income', 'Debt-to-income ratio']

CAT_FEATURE_NAMES = ['Home Ownership', 'Purpose', 'Term', 'Years in current job']

NEW_BINARY_FEATURES = ['Is Loan Big', 'Is Payment High']

SELECTED_FEATURE_NAMES = NUM_FEATURE_NAMES + NEW_BINARY_FEATURES + NEW_FEATURE_NAMES

### Приведение типов для модели CatBoost

In [ ]:
for colname in CAT_FEATURE_NAMES:
    df_train[colname] = pd.Categorical(df_train[colname])
    
df_train[CAT_FEATURE_NAMES].dtypes

### Нормализация данных

In [ ]:
scaler = StandardScaler()

df_norm = df_train.copy()
df_norm[NUM_FEATURE_NAMES] = scaler.fit_transform(df_norm[NUM_FEATURE_NAMES])

df_train = df_norm.copy()
df_train.head()

### Разбиение на train и valid

In [ ]:
X = df_train[SELECTED_FEATURE_NAMES]
y = df_train[TARGET_NAME]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=True, test_size=0.3, random_state=27)

### Балансировка целевой переменной

In [ ]:
df_for_balancing = pd.concat([X, y], axis=1)
df_balanced = balance_df_by_target(df_for_balancing, TARGET_NAME)
    
df_balanced[TARGET_NAME].value_counts()

In [ ]:
X_train = df_balanced.drop(columns=TARGET_NAME)
y_train = df_balanced[TARGET_NAME]

### Обучение и оценка модели на тестовых данных

### Обучение модели

In [ ]:
%%time

model_catb = catb.CatBoostClassifier(l2_leaf_reg=10, silent=True, random_state=27)
model_catb.fit(X_train, y_train)

y_train_pred = final_model.predict(X_train)

print('VALID\n\n' + classification_report(y_train, y_train_pred))

In [ ]:
cv_score_f1 = cross_val_score(model_catb, X, y, scoring=('f1'), cv=KFold(n_splits=10, shuffle=True, random_state=27))
print(cv_score_f1.mean(), cv_score_f1.std())
print(cv_score_f1.mean() - cv_score_f1.std(), cv_score_f1.mean() + cv_score_f1.std())

In [ ]:
cv_score_precision = cross_val_score(model_catb, X, y, scoring=('precision'), cv=KFold(n_splits=10, shuffle=True, random_state=27))
print(cv_score_precision.mean(), cv_score_precision.std())
print(cv_score_precision.mean() - cv_score_precision.std(), cv_score_precision.mean() + cv_score_precision.std())

In [ ]:
cv_score_recall = cross_val_score(model_catb, X, y, scoring=('recall'), cv=KFold(n_splits=10, shuffle=True, random_state=27))
print(cv_score_recall.mean(), cv_score_recall.std())
print(cv_score_recall.mean() - cv_score_recall.std(), cv_score_recall.mean() + cv_score_recall.std())

### Обучение финальной модели

In [ ]:
important_features_top = get_feature_importances(X_train.columns, model_catb.feature_importances_, get_top=15)

**Для обучения финальной модели использую только 15 самых важных признаков датасета**

In [ ]:
%%time

final_model = catb.CatBoostClassifier(l2_leaf_reg=30, silent=True, random_state=27)
final_model.fit(X_train[important_features_top], y_train)

y_train_pred = final_model.predict(X_train[important_features_top])

print('VALID\n\n' + classification_report(y_train, y_train_pred))

In [ ]:
cv_score_f1 = cross_val_score(final_model, X, y, scoring=('f1'), cv=KFold(n_splits=5, shuffle=True, random_state=27))
print(cv_score_f1.mean(), cv_score_f1.std())
print(cv_score_f1.mean() - cv_score_f1.std(), cv_score_f1.mean() + cv_score_f1.std())

In [ ]:
cv_score_precision = cross_val_score(final_model, X, y, scoring=('precision'), cv=KFold(n_splits=5, shuffle=True, random_state=27))
print(cv_score_precision.mean(), cv_score_precision.std())
print(cv_score_precision.mean() - cv_score_precision.std(), cv_score_precision.mean() + cv_score_precision.std())

In [ ]:
cv_score_recall = cross_val_score(final_model, X, y, scoring=('recall'), cv=KFold(n_splits=5, shuffle=True, random_state=27))
print(cv_score_recall.mean(), cv_score_recall.std())
print(cv_score_recall.mean() - cv_score_recall.std(), cv_score_recall.mean() + cv_score_recall.std())

### Подготовка тестового датасета

#### Обзор тестового датасета

In [ ]:
df_test = pd.read_csv(TEST_DATASET_PATH)
df_test.shape

In [ ]:
df_test.head()

In [ ]:
df_test.info()

In [ ]:
df_test.isnull().sum()

**Обработка выбросов. Тестовый датасет**

In [ ]:
df_test.describe()

*Maximum Open Credit*

In [ ]:
df_test.loc[df_test['Maximum Open Credit'] > 1e+07, 'Maximum Open Credit'] = median_open_credit

*Current Loan Amount*

In [ ]:
df_test.loc[df_test['Current Loan Amount'] > 1e+06, 'Current Loan Amount'] = median_loan_ammount

*Credit Score*

In [ ]:
df_test.loc[df_test['Credit Score'] > 751, 'Credit Score'] = df_test.loc[df_test['Credit Score'] > 751, 'Credit Score'] / 10

### Обработка пропусков тестового датасета

##### Annual Income

In [ ]:
for i in df_test.index[df_test['Annual Income'].isnull()]:
    years = df_test.iloc[i, 2]
    df_test.iat[i, 1] = df_train.loc[(df_train['Monthly Debt'] < monthly_debt * 1.2) & 
                                      (df_train['Monthly Debt'] > monthly_debt * 0.8), 'Monthly Debt'].median()

In [ ]:
df_test.loc[df_test['Annual Income'].isnull(), 'Annual Income'] = med_anual_income

##### Years in current job

In [ ]:
df_test.loc[df_test['Years in current job'].isnull(), 'Years in current job'] = job_years

##### Credit Score

In [ ]:
# Использую конструкцию try except, т.к. в словаре credit_score есть не все значения из тестового датасета. 
# Им присваиваю медианное значение
try:
    for i in df_test.index[df_test['Credit Score'].isnull()]:
        cred_history_years_1 = df_test.iloc[i, 5]
        df_test.iat[i, 15] = credit_score[cred_history_years_1]
except KeyError:
    df_test.iat[i, 15] = median_credit_score

In [ ]:
df_test.loc[df_test['Credit Score'].isnull(), 'Credit Score'] = median_credit_score

##### Bankruptcies

In [ ]:
# Аналогичная причина использования try except
try:
    for i in df_test.index[df_test['Bankruptcies'].isnull()]:
        cred_score = df_test.iloc[i, 15]
        df_test.iat[i, 9] = bankruptcies[cred_score]
except KeyError:
    df_test.iat[i, 9] = 0

In [ ]:
df_test.loc[df_test['Bankruptcies'].isnull(), 'Bankruptcies'] = 0

##### Months since last delinquent

Т.к. пропущенных значений больше половины, удаляю данный признак из датасета

In [ ]:
df_test = df_test.drop(columns='Months since last delinquent')

In [ ]:
df_test.isnull().sum()

### Создаю дополнительные признаки

**Is Loan Big - равен 1, если текущая сумма кредита больше медианного значения**

In [ ]:
is_big_loan_test = [1 if el > med_cur_ammount else 0 for el in df_test['Current Loan Amount']]

In [ ]:
df_test['Is Loan Big'] = is_big_loan_test

**Mean Home Ownership Income - средний доход домовладений по типам**

In [ ]:
mean_home_own_income_by_type = [df_test.loc[df_train['Home Ownership'] == el, 'Annual Income'].mean() 
                                for el in home_own_types]

In [ ]:
df_mean_income = pd.DataFrame([home_own_types, mean_home_own_income_by_type], index=['Home Ownership', 'Mean Annual Income'])
df_mean_income = df_mean_income.T

In [ ]:
mean_home_own_income_temp_test = [df_mean_income.loc[df_mean_income['Home Ownership'] == el, 
                                'Mean Annual Income'].tolist() for el in df_test['Home Ownership']]

mean_home_own_income_test = [el[0] for el in mean_home_own_income_temp_test]

In [ ]:
df_test['Mean Home Ownership Income'] = mean_home_own_income_test

**Debt-to-income ratio - Отношение ежемесячного долга домовладения к его ежемесячному доходу**

In [ ]:
df_test['Debt-to-income ratio'] = round(df_test['Monthly Debt'] / (df_test['Annual Income'] / 12), 3)

**Is Payment High - Равен 1, если Debt-to-income ratio более 0.43.**  
Считается, что при более высокой доле обязательств по кредиту относительно общего дохода, становится затруднительно отвечать по своим финансовым обязательствам. Данное обстоятельство ведет к повышению риска просрочки платежей по ним.

In [ ]:
is_payment_high_test = [1 if el > 0.43 else 0 for el in df_test['Debt-to-income ratio']]

In [ ]:
df_test['Is Payment High'] = is_payment_high_test

### Обзор категориальных переменных. Тестовый датасет

In [ ]:
for cat_colname in df_test.select_dtypes(include='object').columns:
    print(str(cat_colname) + '\n\n' + str(df_test[cat_colname].value_counts()) + '\n' + '*' * 100 + '\n')

In [ ]:
df_test['Term Binary'] = df_test['Term'].map({'Short Term':'0', 'Long Term':'1'}).astype(int)

In [ ]:
df_test['Home Ownership digit'] = df_test['Home Ownership'].map({'Home Mortgage':'0', 'Rent':'1',\
                                    'Own Home': '2', 'Have Mortgage': '3'}).astype(int)

In [ ]:
df_test.loc[(df_test['Years in current job'] == '< 1 year') | (df_test['Years in current job'] == '1 year'), 'Years in current job'] = 1
df_test.loc[(df_test['Years in current job'] == '2 years') | (df_test['Years in current job'] == '3 years'), 'Years in current job'] = 2
df_test.loc[(df_test['Years in current job'] == '4 years') | (df_test['Years in current job'] == '5 years'), 'Years in current job'] = 3
df_test.loc[(df_test['Years in current job'] == '6 years') | (df_test['Years in current job'] == '7 years'), 'Years in current job'] = 4
df_test.loc[(df_test['Years in current job'] == '8 years') | (df_test['Years in current job'] == '9 years'), 'Years in current job'] = 5
df_test.loc[df_test['Years in current job'] == '10+ years', 'Years in current job'] = 6

In [ ]:
test_dummies = pd.get_dummies(df_test[obj_columns])
df_test = pd.concat([df_test, test_dummies], axis=1)

In [ ]:
df_test['Purpose_renewable energy'] = 0

### Нормализация данных

In [ ]:
df_norm_test = df_test.copy()
df_norm_test[NUM_FEATURE_NAMES] = scaler.transform(df_norm_test[NUM_FEATURE_NAMES])

df_test = df_norm_test.copy()

### Делаю предсказание на тестовом датасете

In [ ]:
y_test_pred = final_model.predict(df_test[important_features_top])

In [ ]:
df_test_preds = pd.DataFrame(data=y_test_pred, columns=[TARGET_NAME])
df_test_preds.shape

In [ ]:
df_test_preds.head(10)

**Сохраняю получившийся датасет в файл .csv**

In [ ]:
df_test_preds.to_csv(PREDICTED_CREDIT_DEFAULT_PATH, index=False, encoding='utf-8')